# STEPS TO FOLLOW
### Iterate through each entity (except EPUK and EPL) and for each currency
### 1) Hedge between above and below for each entity != entity_ccy using the entity_ccy as the back-to-back
### 2) Add those amounts of entity_ccy to the corresponding above/below of the entity ccy
### 3) Move the excess of each currency (except entity_ccy) from the entity to EPUK, using the entity_ccy as back-to-back
### 4) Add the amount of these movements of entity_ccy to the entity's exposure in the entity_ccy in the above/below (for each entity and EPUK)
### Now, all the currencies for each entity are hedged, except for the entity_ccy. The entity_ccy exposure isn't moved to EPUK
### 6) Now, for EPUK, hedge between above and below for each entity != entity_ccy using the entity_ccy as the back to back
### 7) Hedge the excess of each currency != entity_ccy in the market with EPUK
### 8) For each entity_ccy in each entity_ccy, hedge the excess to the entity_threshold with EPUK (without moving the exposure to EPUK)
### 9) These last hedges need to be moved to EPL so EPUK isn't exposed to that currency, so last step is to do a internal EPUK<->EPL

In [1]:
import pandas as pd
import numpy as np
import queries as qy

In [2]:
import os
from configparser import ConfigParser

def get_config():
    config = ConfigParser()
    config.read(os.path.join("C:/Users/andres.mireles_ebury/Desktop/Projects/FX Exposure/fx_exposure/config.ini"))

    return config

from bq_link import get_bq_link
bq_client, _ = get_bq_link(get_config())

Load all the exposures

In [3]:
balance_date = "2024-04-30"
exposures_entities = bq_client.query(qy.net_exposure.format(date=balance_date)).drop_duplicates()

Set the group threshold and the individual entity threshold (in the future, these can be weighted)

The total exposures by currency

In [4]:
total_exposures = exposures_entities.groupby("currency")[["above_exposure_gbp","above_exposure_local_ccy","below_exposure_gbp","below_exposure_local_ccy","net_exposure_gbp","net_exposure_local_ccy"]].sum()

Use only the entitiies with high exposures

In [5]:
group_threshold = 5e6

In [6]:
large_exposure_currencies = total_exposures[
    (np.abs(total_exposures.above_exposure_gbp)>=group_threshold)|
    (np.abs(total_exposures.below_exposure_gbp)>=group_threshold)|
    (np.abs(total_exposures.net_exposure_gbp)>=group_threshold)
].index

In [7]:
large_exposure_currencies

Index(['AED', 'AUD', 'CAD', 'CHF', 'EUR', 'GBP', 'HKD', 'PLN', 'USD'], dtype='object', name='currency')

In [8]:
exposures_entities = exposures_entities[exposures_entities.currency.isin(large_exposure_currencies)].reset_index(drop=True)

In [9]:
exposures_entities = exposures_entities.set_index(["currency","entity","entity_ccy"])

# THIS PART IS NOT CORRECT (IT DOESN'T ALLOW US TO SEE THE FULL EXPOSURE) AND NOT USEFUL

Only the entities with large exposures to each currency

In [10]:
entity_threshold = group_threshold / (2 * exposures_entities.index.get_level_values(1).unique().shape[0])

In [11]:
# large_exposure_entities_currencies = exposures_entities[
#     (np.abs(exposures_entities.above_exposure_gbp)>=entity_threshold)|
#     (np.abs(exposures_entities.below_exposure_gbp)>=entity_threshold)|
#     (np.abs(exposures_entities.net_exposure_gbp)>=entity_threshold)
# ].index

In [12]:
# exposures_entities = exposures_entities.loc[large_exposure_entities_currencies]

In [13]:
def net_exposure(currency,entity,entity_ccy, above,above_local_ccy,above_entity_ccy,below,below_local_ccy,below_entity_ccy,threshold):

    if above == below == 0 or entity in ("EPUK","EPL") or currency == entity_ccy:
        return above, above_local_ccy, above_entity_ccy, below, below_local_ccy, below_entity_ccy, above+below, above_local_ccy+below_local_ccy, above_entity_ccy+below_entity_ccy, 0, 0, 0, np.nan, np.nan, 0, 0, 0, np.nan
    
    
    # if currency == entity_ccy:
    #     return 

    # Rate to convert gbp to local ccy
    if below != 0:
        rate_to_local_ccy = below_local_ccy / below
        rate_to_entity_ccy = below_entity_ccy / below
    else:
        rate_to_local_ccy = above_local_ccy / above
        rate_to_entity_ccy = above_entity_ccy / above


    # Signs of the below to move the exposure (if below is 0, then just the opposite of the sign of the above)
    if below != 0:
        sign_below = below / abs(below)
    else:
        sign_below = -1 * above / abs(above)

    # The internal change that we need
    internal_change = np.maximum(abs(below) - threshold,0)

    internal_change_local_ccy = internal_change * rate_to_local_ccy
    internal_change_entity_ccy = internal_change * rate_to_entity_ccy

    # The resulting exposures
    new_above = above + sign_below*internal_change
    new_above_local_ccy = above_local_ccy + sign_below*internal_change_local_ccy
    new_above_entity_ccy = above_entity_ccy + sign_below*internal_change_entity_ccy
    new_below = below - sign_below*internal_change
    new_below_local_ccy = below_local_ccy - sign_below*internal_change_local_ccy
    new_below_entity_ccy = below_entity_ccy - sign_below*internal_change_entity_ccy

    # The direction of the internal entity hedges
    below_lhs_rhs = np.where(
        internal_change != 0,
        np.where(sign_below>0,"LHS","RHS"),
        np.nan
    )
    above_lhs_rhs = np.where(
        internal_change != 0,
        np.where(sign_below>0,"RHS","LHS"),
        np.nan
    )

    # What we are going to move from the above of the entity to the above in epuk
    above_to_epuk = np.maximum(abs(new_above) - threshold,0)

    above_to_epuk_local_ccy = above_to_epuk * rate_to_local_ccy
    above_to_epuk_entity_ccy = above_to_epuk * rate_to_entity_ccy
    
    # The direction of the above to epuk hedges (if above = 0, we don't move anything)
    above_to_epuk_lhs_rhs = np.where(
        above_to_epuk != 0,
        np.where(new_above/abs(new_above)>0,"LHS","RHS"),
        np.nan
    )

    # Update the new above of that entity reducing what we move to epuk
    new_above = new_above - np.where(
        new_above!=0,
        new_above/abs(new_above)*above_to_epuk,
        0
    )
    new_above_local_ccy = new_above_local_ccy - np.where(
        new_above!=0,
        new_above_local_ccy/abs(new_above_local_ccy)*above_to_epuk_local_ccy,
        0
    )
    new_above_entity_ccy = new_above_entity_ccy - np.where(
        new_above!=0,
        new_above_entity_ccy/abs(new_above_entity_ccy)*above_to_epuk_entity_ccy,
        0
    )

    # New net
    new_net = new_below + new_above
    new_net_local_ccy = new_below_local_ccy + new_above_local_ccy
    new_net_entity_ccy = new_below_entity_ccy + new_above_entity_ccy
    


    return new_above, new_above_local_ccy, new_above_entity_ccy, new_below, new_below_local_ccy, new_below_entity_ccy, new_net, new_net_local_ccy, new_net_entity_ccy, internal_change, internal_change_local_ccy, internal_change_entity_ccy, above_lhs_rhs, below_lhs_rhs, above_to_epuk, above_to_epuk_local_ccy, above_to_epuk_entity_ccy, above_to_epuk_lhs_rhs


In [14]:
exposures_entities.head()

above_exposure_gbp  above_exposure_local_ccy  \
currency entity entity_ccy                                                 
AED      EPAU   AUD                       0.00                      0.00   
GBP      EPUS   USD                       0.00                      0.00   
EUR      EPAU   AUD                   32530.58                  38096.38   
         EPDIFC USD                 1334502.38                1562831.78   
CAD      EPUS   USD                       0.00                      0.00   

                            above_exposure_entity_ccy  below_exposure_gbp  \
currency entity entity_ccy                                                  
AED      EPAU   AUD                      0.000000e+00         2113.794192   
GBP      EPUS   USD                      0.000000e+00            0.000000   
EUR      EPAU   AUD                      6.272408e+04      -342845.632157   
         EPDIFC USD                      1.669729e+06       217467.713739   
CAD      EPUS   USD                      0.000000e+00            0.000000   

                            below_exposure_local_ccy  \
currency entity entity_ccy                             
AED      EPAU   AUD                          9714.01   
GBP      EPUS   USD                             0.00   
EUR      EPAU   AUD                       -401505.48   
         EPDIFC USD                        254675.78   
CAD      EPUS   USD                             0.00   

                            below_exposure_entity_ccy  net_exposure_gbp  \
currency entity entity_ccy                                                
AED      EPAU   AUD                       4075.737150      2.113794e+03   
GBP      EPUS   USD                          0.000000      0.000000e+00   
EUR      EPAU   AUD                    -661061.839288     -3.103151e+05   
         EPDIFC USD                     272095.603352      1.551970e+06   
CAD      EPUS   USD                          0.000000      0.000000e+00   

                            net_exposure_local_ccy  net_exposure_entity_ccy  
currency entity entity_ccy                                                   
AED      EPAU   AUD                        9714.01             4.075737e+03  
GBP      EPUS   USD                           0.00             0.000000e+00  
EUR      EPAU   AUD                     -363409.10            -5.983378e+05  
         EPDIFC USD                     1817507.56             1.941825e+06  
CAD      EPUS   USD                           0.00             0.000000e+00

## Step 1)

### For each entity except EPUK & EPL, we hedge above/below and move out of each ccy except entity_ccy the remaining exposure

In [15]:
import warnings

warnings.filterwarnings('ignore', category=pd.core.common.SettingWithCopyWarning)

exposures_entities_net = pd.DataFrame()

for entity in exposures_entities.index.get_level_values(1).unique():
    
    exposure_ent = exposures_entities.xs(entity, level=1)

    exposure_ent[
        [
            'new_above_exposure_gbp', 
            'new_above_exposure_local_ccy', 
            'new_above_exposure_entity_ccy', 
            'new_below_exposure_gbp', 
            'new_below_exposure_local_ccy', 
            'new_below_exposure_entity_ccy', 
            'new_net_exposure_gbp', 
            'new_net_exposure_local_ccy', 
            'new_net_exposure_entity_ccy', 
            'internal_change_gbp', 
            'internal_change_local_ccy', 
            'internal_change_entity_ccy', 
            'above_lhs_rhs', 
            'below_lhs_rhs', 
            'above_to_epuk_gbp', 
            'above_to_epuk_local_ccy', 
            'above_to_epuk_entity_ccy', 
            'above_to_epuk_lhs_rhs'
        ]
    ] = exposure_ent.apply(
        lambda row: pd.Series(
            net_exposure(
                row.name[0],
                entity, 
                row.name[1], 
                row['above_exposure_gbp'], 
                row['above_exposure_local_ccy'], 
                row['above_exposure_entity_ccy'], 
                row['below_exposure_gbp'], 
                row['below_exposure_local_ccy'], 
                row['below_exposure_entity_ccy'], 
                entity_threshold
            )
        ),
        axis=1
    )

    exposure_ent["entity"] = entity
    exposure_ent = exposure_ent.reset_index().set_index(["currency","entity","entity_ccy"])
    
    exposures_entities_net = pd.concat([exposures_entities_net,exposure_ent])
    

C:\Users\andres.mireles_ebury\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:60: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\andres.mireles_ebury\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:67: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\andres.mireles_ebury\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:72: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\andres.mireles_ebury\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:77: RuntimeWarning: invalid value encountered in double_scalars


In [16]:
exposures_entities_net[exposures_entities_net.index.get_level_values(1)=="EPBE"][[c for c in exposure_ent.columns if "ccy" not in c]]

,,,above_exposure_gbp,below_exposure_gbp,net_exposure_gbp,new_above_exposure_gbp,new_below_exposure_gbp,new_net_exposure_gbp,internal_change_gbp,above_lhs_rhs,below_lhs_rhs,above_to_epuk_gbp,above_to_epuk_lhs_rhs
currency,entity,entity_ccy,,,,,,,,,,,
USD,EPBE,EUR,5.756789e+06,8.981906e+07,9.557585e+07,1.470588e+05,1.470588e+05,294117.647059,8.967200e+07,RHS,LHS,9.528173e+07,LHS
EUR,EPBE,EUR,6.923909e+07,-6.897756e+07,2.615338e+05,6.923909e+07,-6.897756e+07,261533.793454,0.000000e+00,NaN,NaN,0.000000e+00,NaN
PLN,EPBE,EUR,2.871657e+06,-1.339182e+07,-1.052016e+07,-1.470588e+05,-1.470588e+05,-294117.647059,1.324476e+07,LHS,RHS,1.022604e+07,RHS
AED,EPBE,EUR,1.806255e+04,-1.438481e+05,-1.257856e+05,1.806255e+04,-1.438481e+05,-125785.575036,0.000000e+00,nan,nan,0.000000e+00,nan
CHF,EPBE,EUR,3.507185e+05,-4.712774e+06,-4.362055e+06,-1.470588e+05,-1.470588e+05,-294117.647059,4.565715e+06,LHS,RHS,4.067937e+06,RHS
HKD,EPBE,EUR,0.000000e+00,-2.467729e+03,-2.467729e+03,0.000000e+00,-2.467729e+03,-2467.729208,0.000000e+00,nan,nan,0.000000e+00,nan
GBP,EPBE,EUR,8.275989e+05,7.409683e+07,7.492443e+07,1.470588e+05,1.470588e+05,294117.647059,7.394978e+07,RHS,LHS,7.463032e+07,LHS
CAD,EPBE,EUR,7.970070e+03,1.165937e+05,1.245638e+05,7.970070e+03,1.165937e+05,124563.787083,0.000000e+00,nan,nan,0.000000e+00,nan
AUD,EPBE,EUR,1.574927e+04,-4.826752e+05,-4.669259e+05,-1.470588e+05,-1.470588e+05,-294117.647059,3.356163e+05,LHS,RHS,1.728082e+05,RHS


## Steps 2) & 4) (except EPUK)

In [17]:
exposures_entities_net_2 = pd.DataFrame()

for entity in exposures_entities_net.index.get_level_values(1).unique():

    exposure_ent = exposures_entities_net.xs(entity, level=1)

    if entity not in ("EPUK","EPL"):
        
        # If we don't have already exposure for the entity_ccy, we create it full of 0
        if exposure_ent.index.get_level_values(1).unique().item() not in exposure_ent.index.get_level_values(0):

            exposure_ent = pd.concat(
                [
                    exposure_ent,
                    pd.DataFrame(
                        np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,np.nan,np.nan,0,0,0,np.nan]).reshape(-1,1).T, 
                        index=pd.MultiIndex.from_tuples(
                            [(exposure_ent.index.get_level_values(1).unique().item(),exposure_ent.index.get_level_values(1).unique().item())], 
                            names=exposure_ent.index.names
                            ),
                        columns=exposure_ent.columns
                    )
                ]
            )
            
        # Locate the entity_ccy exposure to add the exposure from the internal changes (to the above and below) and from the hedges to EPUK to the above
        exposure_ent.loc[
            exposure_ent.index.get_level_values(0)==exposure_ent.index.get_level_values(1).unique().item(),
        [
            "new_above_exposure_gbp",
            "new_above_exposure_local_ccy",
            "new_above_exposure_entity_ccy",
        ]] += (np.where(
                exposure_ent[["above_lhs_rhs"]]=="LHS",
                exposure_ent[["internal_change_gbp","internal_change_local_ccy","internal_change_entity_ccy"]],
                -1 * exposure_ent[["internal_change_gbp","internal_change_local_ccy","internal_change_entity_ccy"]]
        ).sum(axis=0) + np.where(
                exposure_ent[["above_to_epuk_lhs_rhs"]]=="LHS",
                exposure_ent[["above_to_epuk_gbp","above_to_epuk_local_ccy","above_to_epuk_entity_ccy"]],
                -1 * exposure_ent[["above_to_epuk_gbp","above_to_epuk_local_ccy","above_to_epuk_entity_ccy"]]
        ).sum(axis=0))

        exposure_ent.loc[
            exposure_ent.index.get_level_values(0)==exposure_ent.index.get_level_values(1).unique().item(),
        [
            "new_below_exposure_gbp",
            "new_below_exposure_local_ccy",
            "new_below_exposure_entity_ccy",
        ]] += np.where(
                exposure_ent[["below_lhs_rhs"]]=="LHS",
                exposure_ent[["internal_change_gbp","internal_change_local_ccy","internal_change_entity_ccy"]],
                -1 * exposure_ent[["internal_change_gbp","internal_change_local_ccy","internal_change_entity_ccy"]]
        ).sum(axis=0)

        # The net exposure of the entity_ccy only changes due to the movement of exposure to EPUK
        exposure_ent.loc[
            exposure_ent.index.get_level_values(0)==exposure_ent.index.get_level_values(1).unique().item(),
        [
            "new_net_exposure_gbp",
            "new_net_exposure_local_ccy",
            "new_net_exposure_entity_ccy",
        ]] += np.where(
                exposure_ent[["above_to_epuk_lhs_rhs"]]=="LHS",
                exposure_ent[["above_to_epuk_gbp","above_to_epuk_local_ccy","above_to_epuk_entity_ccy"]],
                -1 * exposure_ent[["above_to_epuk_gbp","above_to_epuk_local_ccy","above_to_epuk_entity_ccy"]]
        ).sum(axis=0)



    exposure_ent["entity"] = entity
    exposure_ent = exposure_ent.reset_index().set_index(["currency","entity","entity_ccy"])

    exposures_entities_net_2 = pd.concat([exposures_entities_net_2,exposure_ent])



In [18]:
exposures_entities_net_2[exposures_entities_net_2.index.get_level_values(1)=="EPBE"][[c for c in exposure_ent.columns if "ccy" not in c]]

,,,above_exposure_gbp,below_exposure_gbp,net_exposure_gbp,new_above_exposure_gbp,new_below_exposure_gbp,new_net_exposure_gbp,internal_change_gbp,above_lhs_rhs,below_lhs_rhs,above_to_epuk_gbp,above_to_epuk_lhs_rhs
currency,entity,entity_ccy,,,,,,,,,,,
USD,EPBE,EUR,5.756789e+06,8.981906e+07,9.557585e+07,1.470588e+05,1.470588e+05,2.941176e+05,8.967200e+07,RHS,LHS,9.528173e+07,LHS
EUR,EPBE,EUR,6.923909e+07,-6.897756e+07,2.615338e+05,7.920866e+07,7.649813e+07,1.557068e+08,0.000000e+00,NaN,NaN,0.000000e+00,NaN
PLN,EPBE,EUR,2.871657e+06,-1.339182e+07,-1.052016e+07,-1.470588e+05,-1.470588e+05,-2.941176e+05,1.324476e+07,LHS,RHS,1.022604e+07,RHS
AED,EPBE,EUR,1.806255e+04,-1.438481e+05,-1.257856e+05,1.806255e+04,-1.438481e+05,-1.257856e+05,0.000000e+00,nan,nan,0.000000e+00,nan
CHF,EPBE,EUR,3.507185e+05,-4.712774e+06,-4.362055e+06,-1.470588e+05,-1.470588e+05,-2.941176e+05,4.565715e+06,LHS,RHS,4.067937e+06,RHS
HKD,EPBE,EUR,0.000000e+00,-2.467729e+03,-2.467729e+03,0.000000e+00,-2.467729e+03,-2.467729e+03,0.000000e+00,nan,nan,0.000000e+00,nan
GBP,EPBE,EUR,8.275989e+05,7.409683e+07,7.492443e+07,1.470588e+05,1.470588e+05,2.941176e+05,7.394978e+07,RHS,LHS,7.463032e+07,LHS
CAD,EPBE,EUR,7.970070e+03,1.165937e+05,1.245638e+05,7.970070e+03,1.165937e+05,1.245638e+05,0.000000e+00,nan,nan,0.000000e+00,nan
AUD,EPBE,EUR,1.574927e+04,-4.826752e+05,-4.669259e+05,-1.470588e+05,-1.470588e+05,-2.941176e+05,3.356163e+05,LHS,RHS,1.728082e+05,RHS


## Step 3)

In [19]:
exposures_entities_not_entity_ccy = exposures_entities_net_2.copy()
exposures_entities_not_entity_ccy = exposures_entities_not_entity_ccy[exposures_entities_not_entity_ccy.index.get_level_values(0) != exposures_entities_not_entity_ccy.index.get_level_values(2)]

In [20]:
exposures_to_epuk_entities_not_entity_ccy = exposures_entities_not_entity_ccy.reset_index()
exposures_to_epuk_entities_not_entity_ccy[
    [
        "above_to_epuk_gbp",
        "above_to_epuk_local_ccy",
        "above_to_epuk_entity_ccy"
    ]
] = np.where(
    exposures_to_epuk_entities_not_entity_ccy[["above_to_epuk_lhs_rhs"]] == "LHS",
    exposures_to_epuk_entities_not_entity_ccy[
        [
            "above_to_epuk_gbp",
            "above_to_epuk_local_ccy",
            "above_to_epuk_entity_ccy"
        ]
    ],
    -1 * exposures_to_epuk_entities_not_entity_ccy[
        [
            "above_to_epuk_gbp",
            "above_to_epuk_local_ccy",
            "above_to_epuk_entity_ccy"
        ]
    ]
)

# The change that we need to add by currency
exposures_to_epuk = exposures_to_epuk_entities_not_entity_ccy.groupby("currency")[[
            "above_to_epuk_gbp",
            "above_to_epuk_local_ccy",
            "above_to_epuk_entity_ccy"
]].sum().reset_index()

# Add these required fields
exposures_to_epuk[["entity","entity_ccy"]] = ["EPUK","GBP"]

exposures_to_epuk = exposures_to_epuk.set_index(["currency","entity","entity_ccy"])

# Add the exposures to EPUK (to the above and to the net)
exposures_entities_net_3 = exposures_entities_net_2.copy()
exposures_entities_net_3.loc[
    ((exposures_entities_net_2.index.get_level_values(1)=="EPUK")&(exposures_entities_net_2.index.get_level_values(2)=="GBP")),
    ["new_above_exposure_gbp","new_above_exposure_local_ccy","new_above_exposure_entity_ccy"]] += exposures_to_epuk.values
exposures_entities_net_3.loc[
    ((exposures_entities_net_2.index.get_level_values(1)=="EPUK")&(exposures_entities_net_2.index.get_level_values(2)=="GBP")),
    ["new_net_exposure_gbp","new_net_exposure_local_ccy","new_net_exposure_entity_ccy"]] += exposures_to_epuk.values

In [21]:
exposures_entities_net_3[exposures_entities_net_3.index.get_level_values(1)=="EPUK"][[c for c in exposure_ent.columns if "ccy" not in c]]

,,,above_exposure_gbp,below_exposure_gbp,net_exposure_gbp,new_above_exposure_gbp,new_below_exposure_gbp,new_net_exposure_gbp,internal_change_gbp,above_lhs_rhs,below_lhs_rhs,above_to_epuk_gbp,above_to_epuk_lhs_rhs
currency,entity,entity_ccy,,,,,,,,,,,
AUD,EPUK,GBP,-2.590651e+07,1.365527e+07,-1.225125e+07,-2.785342e+07,1.365527e+07,-1.419815e+07,0.0,NaN,NaN,0.0,NaN
HKD,EPUK,GBP,3.183231e-12,-2.626989e+06,-2.626989e+06,3.791744e+05,-2.626989e+06,-2.247815e+06,0.0,NaN,NaN,0.0,NaN
PLN,EPUK,GBP,-9.311577e+05,5.686571e+05,-3.625006e+05,-9.311577e+05,5.686571e+05,-3.625006e+05,0.0,NaN,NaN,0.0,NaN
AED,EPUK,GBP,-2.200000e-01,-9.873862e+06,-9.873862e+06,-4.067938e+06,-9.873862e+06,-1.394180e+07,0.0,NaN,NaN,0.0,NaN
GBP,EPUK,GBP,1.407771e+08,-2.020161e+08,-6.123892e+07,1.224925e+08,-2.020161e+08,-7.952361e+07,0.0,NaN,NaN,0.0,NaN
USD,EPUK,GBP,-2.529958e+06,1.179007e+07,9.260114e+06,7.416728e+07,1.179007e+07,8.595736e+07,0.0,NaN,NaN,0.0,NaN
EUR,EPUK,GBP,-8.015238e+07,1.409674e+08,6.081498e+07,-8.015238e+07,1.409674e+08,6.081498e+07,0.0,NaN,NaN,0.0,NaN
CAD,EPUK,GBP,5.865700e+02,-3.635368e+06,-3.634782e+06,-1.022545e+07,-3.635368e+06,-1.386082e+07,0.0,NaN,NaN,0.0,NaN
CHF,EPUK,GBP,-4.355492e+06,3.180092e+06,-1.175400e+06,7.967630e+07,3.180092e+06,8.285640e+07,0.0,NaN,NaN,0.0,NaN


## Step 4) (for EPUK)

In [23]:
exposures_to_epuk_entity_ccy

,,,above_to_epuk_gbp,above_to_epuk_local_ccy,above_to_epuk_entity_ccy
currency,entity,entity_ccy,,,
AUD,EPUK,GBP,3.485161e+05,4.373608e+05,6.719954e+05
CAD,EPUK,GBP,2.934466e+06,4.392936e+06,5.047905e+06
CHF,EPUK,GBP,6.590289e+06,7.966680e+06,7.565494e+06
CLP,EPUK,GBP,0.000000e+00,0.000000e+00,0.000000e+00
CNY,EPUK,GBP,0.000000e+00,0.000000e+00,0.000000e+00
EUR,EPUK,GBP,1.554453e+08,1.370304e+08,1.820415e+08
GBP,EPUK,GBP,-4.103730e+07,-4.878783e+07,-4.103730e+07
HKD,EPUK,GBP,3.697979e+06,2.867482e+06,3.618777e+07
MXN,EPUK,GBP,0.000000e+00,0.000000e+00,0.000000e+00


In [22]:
exposures_to_epuk_entity_ccy = exposures_to_epuk_entities_not_entity_ccy.groupby(by=["entity_ccy"])[[
            "above_to_epuk_gbp",
            "above_to_epuk_local_ccy",
            "above_to_epuk_entity_ccy"
]].sum().reset_index().rename(columns={"entity_ccy":"currency"})

exposures_to_epuk_entity_ccy[["entity","entity_ccy"]] = ["EPUK","GBP"]

exposures_to_epuk_entity_ccy = exposures_to_epuk_entity_ccy.set_index(["currency","entity","entity_ccy"])

# Add the exposures to EPUK (to the above and to the net)
exposures_entities_net_4 = exposures_entities_net_3.copy()
exposures_entities_net_4.loc[
    ((exposures_entities_net_3.index.get_level_values(1)=="EPUK")&(exposures_entities_net_3.index.get_level_values(2)=="GBP")),
    ["new_above_exposure_gbp","new_above_exposure_local_ccy","new_above_exposure_entity_ccy"]] += exposures_to_epuk_entity_ccy.values
exposures_entities_net_4.loc[
    ((exposures_entities_net_3.index.get_level_values(1)=="EPUK")&(exposures_entities_net_3.index.get_level_values(2)=="GBP")),
    ["new_net_exposure_gbp","new_net_exposure_local_ccy","new_net_exposure_entity_ccy"]] += exposures_to_epuk_entity_ccy.values

ValueError: Unable to coerce to DataFrame, shape must be (9, 3): given (11, 3)

In [ ]:
exposures_entities_net_4.loc[
    ((exposures_entities_net_3.index.get_level_values(1)=="EPUK")&(exposures_entities_net_3.index.get_level_values(2)=="GBP")),
    ["new_above_exposure_gbp","new_above_exposure_local_ccy","new_above_exposure_entity_ccy"]]

In [ ]:
exposures_entities_net_4.net_exposure_gbp.sum()

In [ ]:
exposures_entities_net_4.new_net_exposure_gbp.sum()

## Step 5)

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1,2,figsize=(15,5))

axs[0].bar(x=exposures_entities["above_exposure_gbp"].index.get_level_values(0),height=exposures_entities["above_exposure_gbp"])
axs[0].bar(x=exposures_entities["below_exposure_gbp"].index.get_level_values(0),height=exposures_entities["below_exposure_gbp"])
axs[0].legend(["Above Exposure (GBP)","Below Exposure (GBP)"])
axs[0].set_title("Pre Exposure")
axs[0].set_xticklabels(exposures_entities.index.get_level_values(0), rotation=45)

axs[1].bar(x=exposures_entities["new_above_gbp"].index.get_level_values(0),height=exposures_entities["new_above_gbp"])
axs[1].bar(x=exposures_entities["new_below_gbp"].index.get_level_values(0),height=exposures_entities["new_below_gbp"])
axs[1].legend(["New Above Exposure (GBP)","New Below Exposure (GBP)"])
axs[1].set_title("New Exposure")
axs[1].set_xticklabels(exposures_entities.index.get_level_values(0), rotation=45)
plt.tight_layout()

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1,2,figsize=(15,5))

axs[0].bar(x=exposures_entities["net_exposure_gbp"].index.get_level_values(0),height=exposures_entities["net_exposure_gbp"],color="green")
axs[0].legend(["Net Exposure (GBP)"])
axs[0].set_title("Pre Exposure")
axs[0].set_xticklabels(exposures_entities.index.get_level_values(0), rotation=45)

axs[1].bar(x=exposures_entities["new_net_gbp"].index.get_level_values(0),height=exposures_entities["new_net_gbp"],color="green")
axs[1].legend(["New Net Exposure (GBP)"])
axs[1].set_title("New Exposure")
axs[1].set_xticklabels(exposures_entities.index.get_level_values(0), rotation=45)
plt.show()